## Librerias

In [2]:
library(FactoClass)
library(bigrquery)
library(dplyr)
library(knitr)
library(DT)
library(DBI)


Adjuntando el paquete: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## Parametros

In [6]:
# Proyecto de GCP
project <- "co-keralty-costomedico"

# Query para traer la data de diabetes
query <- "SELECT * FROM Diabetes_avicena_survival.diabetes_final"

# Configuracion de la conexion a BigQuery
con <- dbConnect(bigrquery::bigquery(),
                 project = project,
                 dataset = "Diabetes_avicena_survival",
                 location = "us-east1")

# Columnas cualitativas
variables_numericas <- c("year",
                         "month",
                         "edad",
                         "peso",
                         "talla",
                         "HDL",
                         "LDL",
                         "trigliceridos",
                         "perimetro_abdominal",
                         "time_to_event")

# Columnas categoricas
variables_factores <- c("genero_paciente",
                        "raza_paciente",
                        "nivel_academico_paciente",
                        "ant_cardio",
                        "med_hipertension",
                        "ant_familiar_dm",
                        "hace_ejercicio",
                        "diabetes")

## Carga de datos

In [5]:
# Traer la data desde BigQuery
data <- dbGetQuery(con, query)

In [8]:
# Realizar cambios en cada una de las columnas de la data
data <- data %>% mutate(across(all_of(variables_factores), as.factor))
data <- data %>% mutate(across(all_of(variables_numericas), as.numeric))

# Vista de los cambios realizados
str(data)

tibble [9,024 × 20] (S3: tbl_df/tbl/data.frame)
 $ numero_identificacion_paciente: chr [1:9024] "1007107070" "26151819" "93396738" "43524938" ...
 $ year                          : num [1:9024] 2022 2022 2022 2024 2024 ...
 $ month                         : num [1:9024] 8 8 8 1 1 5 5 9 9 2 ...
 $ fecha                         : Date[1:9024], format: "2022-08-01" "2022-08-01" ...
 $ edad                          : num [1:9024] 19 70 47 56 50 44 45 43 54 31 ...
 $ peso                          : num [1:9024] 56 82 85.2 56 91 ...
 $ talla                         : num [1:9024] 1.61 1.59 1.6 1.55 1.75 1.63 1.7 1.69 1.8 1.57 ...
 $ HDL                           : num [1:9024] 56 45.3 44 51 38 45.5 46 30 51.6 33.4 ...
 $ LDL                           : num [1:9024] 99 158 174 143 151 ...
 $ trigliceridos                 : num [1:9024] 95 238 90.1 254.7 211 ...
 $ perimetro_abdominal           : num [1:9024] 75 96 103 74.5 95 ...
 $ genero_paciente               : Factor w/ 2 levels "Femenino

## Caracterizacion

In [9]:
print("Coeficientes de correlacion entre las variables continuas y Diabetes")
kable(centroids(data[, variables_numericas[3:length(variables_numericas)]],
                data$diabetes)$cr * 100,
      digits = 2)

[1] "Coeficientes de correlacion entre las variables continuas y Diabetes"




| edad| peso| talla|  HDL|  LDL| trigliceridos| perimetro_abdominal| time_to_event|
|----:|----:|-----:|----:|----:|-------------:|-------------------:|-------------:|
| 0.03|  0.2|  0.02| 0.22| 0.13|          0.16|                0.33|          0.62|

In [10]:
print("Valores test para caracterización de variables cuantitativas y Diabetes")
cluster.carac(data[, variables_numericas[3:length(variables_numericas)]],
              data$diabetes,
              tipo.v = "co",
              v.lim = 2,
              dn = 4,
              dm = 4,
              neg = TRUE)

[1] "Valores test para caracterización de variables cuantitativas y Diabetes"


class: 0
                    Test.Value Class.Mean Frequency Global.Mean
time_to_event           7.4718    15.8299      7524     15.4930
HDL                     4.4670    48.6059      7524     48.3305
LDL                     3.4252   113.9575      7524    113.3086
trigliceridos          -3.7673   152.6679      7524    154.3656
peso                   -4.2994    74.1305      7524     74.4640
perimetro_abdominal    -5.4365    93.7123      7524     94.0424
------------------------------------------------------------ 
class: 1
                    Test.Value Class.Mean Frequency Global.Mean
perimetro_abdominal     5.4365    95.6977      1500     94.0424
peso                    4.2994    76.1372      1500     74.4640
trigliceridos           3.7673   162.8816      1500    154.3656
LDL                    -3.4252   110.0533      1500    113.3086
HDL                    -4.4670    46.9491      1500     48.3305
time_to_event          -7.4718    13.8033      1500     15.4930

In [11]:
print("Pruebas Test y Chi2 para vars cualitativas y Diabetes")
kable(chisq.carac(data[, variables_factores], data$diabetes, decr = FALSE),
      digits = 8)

[1] "Pruebas Test y Chi2 para vars cualitativas y Diabetes"




|                         |         chi2| dfr|       pval|        tval|       phi2|
|:------------------------|------------:|---:|----------:|-----------:|----------:|
|genero_paciente          |    6.7956290|   1| 0.00913813|  2.35997300| 0.00075306|
|raza_paciente            |    6.1002929|   6| 0.41204958|  0.22227583| 0.00067601|
|nivel_academico_paciente |   20.4571028|  12| 0.05891830|  1.56391897| 0.00226697|
|ant_cardio               |   32.0785168|   1| 0.00000001|  5.54370875| 0.00355480|
|med_hipertension         |    0.9304823|   1| 0.33473763|  0.42686830| 0.00010311|
|ant_familiar_dm          |    0.4542161|   1| 0.50033957| -0.00085118| 0.00005033|
|hace_ejercicio           |    0.6896452|   3| 0.87563676| -1.15344818| 0.00007642|
|diabetes                 | 9016.7860803|   1| 0.00000000|         Inf| 0.99920059|

In [12]:
# Valores test para caracterización de diabetes
# según las variables cualitativas
data_prueba <- data.frame(data)
test_values <- cluster.carac(data_prueba[,
                               variables_factores[0:
                                                    (length(variables_factores)
                                                     - 1)]
                             ],
                             data_prueba$diabetes)

In [14]:
print("Valores test para las categorias contra Diabetes")
print(test_values)

[1] "Valores test para las categorias contra Diabetes"
class: 0
                                     Test.Value p.Value Class.Cat Cat.Class
ant_cardio.1                              5.786   0.000      85.6      52.0
genero_paciente.Masculino                 2.844   0.004      84.7      39.5
nivel_academico_paciente.Profesional      2.777   0.005      86.1      12.3
raza_paciente.Mestizo                     2.081   0.037      86.8       4.2
raza_paciente.Otros                      -2.273   0.023      83.2      93.4
genero_paciente.Femenino                 -2.844   0.004      82.6      60.5
nivel_academico_paciente.Normalista      -3.121   0.002      78.9       6.3
ant_cardio.0                             -5.786   0.000      81.1      48.0
                                     Global Weight
ant_cardio.1                           50.6   4570
genero_paciente.Masculino              38.9   3510
nivel_academico_paciente.Profesional   11.9   1078
raza_paciente.Mestizo                   4.0    3